In [1]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import random

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.intercept_)
print(reg.coef_)

151.88331005254167
[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

### Creating our Mini Batch Gradient Descent Class

In [8]:
class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        

    def fit(self,X_train,y_train):
        # initialize coefficient
        self.intercept_ = 0
        # columns coefficient initalize with 1
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            # update all the coef and the intercept for each batch

            for j in range(int(X_train.shape[0]/self.batch_size)):
                # generating batch size equal random value between 0 & number of rows
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                # prediction of idx row
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                # updating intercept
                intercept_slope = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_slope)
                
                # updating coef
                coef_slope = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_slope)
        
        print(self.intercept_,self.coef_)
    

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [9]:
mbgdr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)

In [10]:
mbgdr.fit(X_train,y_train)

151.12985215440764 [  26.4408508  -140.51314579  451.85267938  299.85928676  -20.30552789
  -88.02377548 -190.19665744  116.03237067  407.2327787   110.59570667]


In [11]:
y_pred = mbgdr.predict(X_test)

In [12]:
r2_score(y_test,y_pred)

0.4537593830986182

#### Apply sklearn SGDRegressor class

In [13]:
from sklearn.linear_model import SGDRegressor

In [14]:
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [15]:
batch_size = 35
epoch = 100

for i in range(epoch):
    idx = random.sample(range(X_train.shape[0]),batch_size)
    # partial_fit() ---> take subset of X_train & y_train
    # Perform one epoch of stochastic gradient descent on given samples.
    sgd.partial_fit(X_train[idx],y_train[idx])

In [16]:
X_train.shape[0]/35

10.085714285714285

In [17]:
sgd.coef_

array([  75.02353548,  -64.3990338 ,  345.36126923,  246.8273639 ,
         23.22748514,  -23.48845394, -178.54644522,  135.73482025,
        325.7487178 ,  126.32877359])

In [18]:
sgd.intercept_

array([147.31926362])

In [19]:
y_pred = sgd.predict(X_test)

In [20]:
r2_score(y_test,y_pred)

0.4253922586478559